In [9]:


    
class Solve_Problem:
    ''' Pass date for the get_ops function. It gets the operations table on that particullar date from the sqlite db.
    Convert them and other necessary tables into pandas df. Those dataframes are passed into pyomo as decision variables and contsraints.
    Solver solves the problem. If the solution does not exclude any existing operation from the solution, it approves the last application and updates the table in the db.
    Otherwise, it returns only a warning that the registration can be done on that particular date. The table is not modified.'''
    
    def _init_(self):
        pass
    def get_ops(self, user_entries, departments_info, operation_rooms_info):
        
        # operations
        basket = pd.DataFrame(user_entries, columns=['id','doctor','operation_date','department_name','operation_duration','operation_urgency','operation_room'])
        basket.set_index('id')

        #departments
        dep_df=pd.DataFrame(departments_info,columns=['id','department_name','department_capacity','date'])
        dep_df.set_index('id')
        dep_df['department_capacity']=dep_df['department_capacity'].astype(int)
        
        # operation rooms
        op_df=pd.DataFrame(operation_rooms_info,columns=['id','room_name','room_capacity','date'])
        op_df.set_index('id')
        op_df['room_capacity']=op_df['room_capacity'].astype(int)
        
        # multi-knapsack, integer divisible

        mdl = pyo.ConcreteModel()

        # make sure id is also index in database user__entries
        mdl.invs = pyo.Set(initialize=list(zip(basket.index, basket["department_name"])))
        mdl.bins = pyo.Set(initialize=list(op_df.room_name)) ## list of operations room from db
        mdl.deps = pyo.Set(initialize=list(dep_df.department_name)) # list of departments from db

        # params
        mdl.value   = pyo.Param(mdl.invs, initialize= {(i,row["department_name"]):row["operation_urgency"] for i,row in basket.iterrows()} )
        mdl.weight  = pyo.Param(mdl.invs, initialize= {(i,row["department_name"]):row["operation_duration"] for i,row in basket.iterrows()})
        mdl.bin_cap = pyo.Param(mdl.bins, initialize= {row["room_name"]:row["room_capacity"] for i,row in op_df.iterrows()} )
        mdl.dep_cap = pyo.Param(mdl.deps, initialize= {row["department_name"]:row["department_capacity"] for i,row in dep_df.iterrows()}, mutable=True)



        # vars
        mdl.X = pyo.Var(mdl.invs, mdl.bins, within=pyo.Binary)     # the amount from invoice i in bin j



        ### Objective ###

        mdl.OBJ = pyo.Objective(expr=sum(mdl.X[i, b]*mdl.value[i] for 
                                i in mdl.invs for
                                b in mdl.bins), sense=pyo.maximize)


        ### constraints ###

        # don't overstuff bin
        def bin_limit(self, b):
            return sum(mdl.X[i, b]*mdl.weight[i] for i in mdl.invs) <= mdl.bin_cap[b]
        mdl.bin_limit = pyo.Constraint(mdl.bins, rule=bin_limit)

        # one_item can be only in a single op room.
        def one_item(self, i,d):
            return sum(mdl.X[i,d,b] for b in mdl.bins) <=1
        mdl.one_item = pyo.Constraint(mdl.invs, rule=one_item)



        # department limits

        mdl.dep_limits=pyo.ConstraintList()

        for d in mdl.deps:
            d_list=[]
            for i in mdl.X:
                if d==i[1]:
                    d_list.append(i)    
            mdl.dep_limits.add(expr=(sum(mdl.X[i]*mdl.weight[i[:2]] for i in d_list)<=mdl.dep_cap[d])) 



        # solve it...
        solver = pyo.SolverFactory('glpk')
        results = solver.solve(mdl)

        
        # save the output into dictionary
        dic=dict()

        for i in mdl.X:
            if pyo.value(mdl.X[i])==1:
                dic[i[0]]=i[2]

        
        if basket.index.isin(list(dic.keys())).all():
            basket['operation_room'] = basket.index.map(dic) 
            return basket

        else:
            return('not possible')

In [17]:

import pyomo.environ as pyo #install pyomo
# conda install -c conda-forge pyomo
# install glpk 
# conda install -c conda-forge glpk
import pandas as pd
import sqlite3
date='''"2022-02-19 00:00:00.000000"'''
con = sqlite3.connect('/Users/kultiginbozdemir/Documents/GitHub/op_room_opt/WebApp/test2.db') # change the path

In [24]:
query = ''.join(['select * FROM user__entries WHERE operation_date==',str(date)])# change name of the table.  save them with date in db. 
cur=con.cursor()
cur.execute(query)
user_entries=cur.fetchall()
user_entries

[('Department_5-87ca82a9-b5f6-489c-9384-d75d2c632a47',
  'Jack',
  '2022-02-19 00:00:00.000000',
  'Department_5',
  70,
  7,
  None),
 ('Department_4-440ad8fa-4580-49cc-a12b-faf0a31af41a',
  'John Cena',
  '2022-02-19 00:00:00.000000',
  'Department_4',
  90,
  5,
  None),
 ('Department_1-20e8e301-1cdd-4b29-ac26-72a7376b5c22',
  'Kurt Boz',
  '2022-02-19 00:00:00.000000',
  'Department_1',
  100,
  10,
  None),
 ('Department_2-4dd8a4b8-978d-498e-a1bb-f07897abd374',
  'Shirvan Hashimov',
  '2022-02-19 00:00:00.000000',
  'Department_2',
  200,
  8,
  None),
 ('Department_2-eefb1c4c-ccb2-4611-bd4c-a1a420bda81c',
  'Dekre dss',
  '2022-02-19 00:00:00.000000',
  'Department_2',
  35,
  3,
  None)]

In [25]:
query = ''.join(['select * FROM operation_rooms__info WHERE date==',str(date)])# change name of the table.  save them with date in db. 
cur=con.cursor()
cur.execute(query)
operation_rooms_info=cur.fetchall()
operation_rooms_info

[('Room_4-ca1e87c1-38a8-44e5-886f-91a550a5667e',
  'Room_4',
  450,
  '2022-02-19 00:00:00.000000'),
 ('Room_3-837b247d-1031-406b-a319-6f8f1bc4a456',
  'Room_3',
  500,
  '2022-02-19 00:00:00.000000'),
 ('Room_2-2d24cbf5-fad8-47c8-b4fb-e1faf838aa4f',
  'Room_2',
  300,
  '2022-02-19 00:00:00.000000'),
 ('Room_5-c62b1831-2dfa-4fb1-abff-8d04e35f1538',
  'Room_5',
  270,
  '2022-02-19 00:00:00.000000'),
 ('Room_1-db87dcca-44cf-452e-abe3-914fffde47ea',
  'Room_1',
  500,
  '2022-02-19 00:00:00.000000')]

In [26]:
query = ''.join(['select * FROM department__info WHERE date==',str(date)])# change name of the table.  save them with date in db. 
cur=con.cursor()
cur.execute(query)
departments_info=cur.fetchall()
departments_info


[('Department_1-087d488d-6966-489f-9681-c2b592e2656b',
  'Department_1',
  350,
  '2022-02-19 00:00:00.000000'),
 ('Department_2-3f779c71-082c-44b2-ad6e-427d2d077259',
  'Department_2',
  450,
  '2022-02-19 00:00:00.000000'),
 ('Department_3-9653c8f0-3648-4c95-9247-32415191fda4',
  'Department_3',
  200,
  '2022-02-19 00:00:00.000000'),
 ('Department_4-de9c7093-68ab-4532-b572-b731c0c70d96',
  'Department_4',
  290,
  '2022-02-19 00:00:00.000000'),
 ('Department_5-e593f3c4-41c8-404a-bf1a-0bacaf364fbb',
  'Department_5',
  410,
  '2022-02-19 00:00:00.000000')]

In [27]:
a=Solve_Problem()

In [28]:
a.get_ops(user_entries, departments_info, operation_rooms_info)

,id,doctor,operation_date,department_name,operation_duration,operation_urgency,operation_room
0,Department_5-87ca82a9-b5f6-489c-9384-d75d2c632a47,Jack,2022-02-19 00:00:00.000000,Department_5,70,7,Room_1
1,Department_4-440ad8fa-4580-49cc-a12b-faf0a31af41a,John Cena,2022-02-19 00:00:00.000000,Department_4,90,5,Room_5
2,Department_1-20e8e301-1cdd-4b29-ac26-72a7376b5c22,Kurt Boz,2022-02-19 00:00:00.000000,Department_1,100,10,Room_1
3,Department_2-4dd8a4b8-978d-498e-a1bb-f07897abd374,Shirvan Hashimov,2022-02-19 00:00:00.000000,Department_2,200,8,Room_1
4,Department_2-eefb1c4c-ccb2-4611-bd4c-a1a420bda81c,Dekre dss,2022-02-19 00:00:00.000000,Department_2,35,3,Room_1


In [29]:
con.commit()
con.close()